In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# source_predict_1 = pd.read_csv('./source_predict_1.csv')
source_predict_2 = pd.read_csv('./source_predict_2.csv')
# # source_predict_3 = pd.read_csv('./source_predict_3.csv')
# source_predict_7 = pd.read_csv('./source_predict_7.csv')
pd.set_option('display.max_columns',None)

In [3]:
source_predict_2['source_run_date'] = pd.to_datetime(source_predict_2['source_run_date'])
source_predict_2.drop_duplicates(inplace=True)

In [6]:
source_predict_2.head()

,channel_id,source_id,game_id,media_id,source_run_date_amount,source_run_date,source_30_roi_true,source_30_roi_predict,source_true_type,source_type,add_time,up_time,platform
0,6770,268577,1000840,16,12416.12,2020-07-01,NaN,0.305148,NaN,0,2020-9-3 12:56:48,2020-9-3 21:00:09,1
1,6770,268577,1000840,16,7529.87,2020-07-02,NaN,0.237650,NaN,0,2020-9-3 12:06:50,2020-9-3 21:09:43,1
2,6770,268577,1000840,16,4060.37,2020-07-03,NaN,0.426720,NaN,2,2020-9-3 14:25:26,2020-9-3 21:19:07,1
3,6770,268577,1000840,16,2456.62,2020-07-04,NaN,0.657029,NaN,2,2020-9-3 13:45:13,2020-9-3 21:25:36,1
4,6770,268577,1000840,16,2429.90,2020-07-05,NaN,0.958052,NaN,2,2020-9-3 15:14:50,2020-9-3 15:14:50,1


In [7]:
df_0 = source_predict_2[source_predict_2['source_type'] == 0]
df_0.sort_values('source_run_date',inplace=True)
df_0_1 = df_0.drop_duplicates(subset=['channel_id','source_id'],keep='first')

In [8]:
df_0_1 = df_0_1[['channel_id','source_id','source_run_date']]
df_0_1['time_0'] = df_0_1['source_run_date'].astype(np.str).str[-2:].astype(np.int64)
df_0_1.drop('source_run_date',axis=1,inplace=True)

In [9]:
data = pd.read_csv('./data.csv',index_col=0)

In [10]:
data['channel_id'] = data['channel_id'].astype(np.int64)
data['source_id'] = data['source_id'].astype(np.int64)

In [11]:
data_merge = pd.merge(data,df_0_1,on=['channel_id','source_id'],how='left',validate='many_to_one')

In [12]:
data_merge.dropna(how='any',inplace=True)
data_merge['time_0'] = data_merge['time_0'].astype(np.int64)

In [13]:
data_merge['istrue'] = (data_merge['time']>data_merge['time_0'])

In [14]:
data_merge_true=data_merge[data_merge['istrue']==1]

In [15]:
data_merge_true['pay_sum'].sum()/data_merge_true['amount'].sum()

0.37934033665917866

In [16]:
data_merge_true['amount'].sum()

5689729.64

In [17]:
data_merge_true['pay_sum'].sum()

2158343.957137307

In [18]:
data['amount'].sum()

15267613.02

In [19]:
data['pay_sum'].sum()

5244943.955602777

In [20]:
data['pay_sum'].sum()/data['amount'].sum()

0.343533985877956

In [21]:
data_merge_true.head()

,channel_id,source_id,30ROI,amount,time,pay_sum,time_0,istrue
1210,20143,322525,0.000000,393.25,2,0.0,1,True
1215,20646,322830,38.471290,851.96,2,32776.0,1,True
1221,20754,322961,0.000000,215.31,2,0.0,1,True
1223,21116,322696,0.332423,1028.81,2,342.0,1,True
1230,21116,322896,0.000000,379.24,2,0.0,1,True


In [22]:
groups = data_merge_true.groupby(['channel_id','source_id'])
temp = groups['amount'].agg([('source_amount_sum','sum')]).reset_index()
m = pd.merge(data_merge_true,temp,on=['channel_id','source_id'],how='left')
temp = groups['pay_sum'].agg([('source_pay_sum','sum')]).reset_index()
m = pd.merge(m,temp,on=['channel_id','source_id'],how='left')

In [24]:
m = m[['channel_id','source_id','source_amount_sum','source_pay_sum']].drop_duplicates()

In [26]:
m['ROE'] = m['source_pay_sum']/m['source_amount_sum']

In [27]:
m.head()

,channel_id,source_id,source_amount_sum,source_pay_sum,ROE
0,20143,322525,496.32,0.0,0.000000
1,20646,322830,851.96,32776.0,38.471290
2,20754,322961,215.31,0.0,0.000000
3,21116,322696,1028.81,342.0,0.332423
4,21116,322896,379.24,0.0,0.000000


In [29]:
m = m.sort_values('source_pay_sum',ascending=False)

In [38]:
m.head(20)

,channel_id,source_id,source_amount_sum,source_pay_sum,ROE
69,20754,322678,90788.97,145000.000327,1.597110
5161,20953,330725,117094.86,124935.999659,1.066964
6203,20953,330590,45867.09,90001.999910,1.962235
60,20754,322312,66979.53,87621.997253,1.308191
6236,20953,332078,202594.09,82941.999905,0.409400
2978,20144,326813,50843.27,81717.999999,1.607253
62,20754,321941,61984.32,77947.994003,1.257544
63,20755,322389,89401.98,74547.999965,0.833852
465,21116,323734,67099.17,70525.999929,1.051071
4695,21117,329767,79112.42,63427.999989,0.801745


In [35]:
m.to_csv('./df.csv',index=None)

In [37]:
data_merge_true[data_merge_true['source_id']==322830]

,channel_id,source_id,30ROI,amount,time,pay_sum,time_0,istrue
1215,20646,322830,38.47129,851.96,2,32776.0,1,True
3890,20646,322830,0.00000,0.00,3,0.0,1,True
5208,20646,322830,0.00000,0.00,4,0.0,1,True
6468,20646,322830,0.00000,0.00,5,0.0,1,True
7680,20646,322830,0.00000,0.00,6,0.0,1,True
10266,20646,322830,0.00000,0.00,8,0.0,1,True
11767,20646,322830,0.00000,0.00,9,0.0,1,True
13256,20646,322830,0.00000,0.00,10,0.0,1,True
14657,20646,322830,0.00000,0.00,11,0.0,1,True
15996,20646,322830,0.00000,0.00,12,0.0,1,True


In [36]:
data[data['source_id']==322830]

,channel_id,source_id,30ROI,amount,time,pay_sum
305,20646,322830,0.00000,1.05,1,0.0
11,20646,322830,38.47129,851.96,2,32776.0
1696,20646,322830,0.00000,0.00,3,0.0
1568,20646,322830,0.00000,0.00,4,0.0
1482,20646,322830,0.00000,0.00,5,0.0
1407,20646,322830,0.00000,0.00,6,0.0
1494,20646,322830,0.00000,0.00,8,0.0
1633,20646,322830,0.00000,0.00,9,0.0
1558,20646,322830,0.00000,0.00,10,0.0
1488,20646,322830,0.00000,0.00,11,0.0
